In [1]:
!pip install torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import precision_score, recall_score, f1_score
import time

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Convolutional block with Batch Normalization and Leaky ReLU
def conv_batch(in_channels, out_channels, kernel_size, stride, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.1)
    )

In [5]:
# Dark Residual Block
class DarkResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(DarkResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            conv_batch(in_channels, in_channels // 2, kernel_size=1, stride=1, padding=0),
            conv_batch(in_channels // 2, in_channels, kernel_size=3, stride=1)
        )

    def forward(self, x):
        return x + self.layer(x)

In [6]:
# Making a layer with multiple residual blocks
def make_layer(block, in_channels, num_blocks):
    layers = []
    for _ in range(num_blocks):
        layers.append(block(in_channels))
    return nn.Sequential(*layers)

In [27]:
# Define the Darknet-53 Model
class Darknet53(nn.Module):
    def __init__(self, num_classes=31):
        super(Darknet53, self).__init__()
        self.conv1 = conv_batch(3, 32, kernel_size=3, stride=1)
        self.conv2 = conv_batch(32, 64, kernel_size=3, stride=2)
        self.residual_block1 = make_layer(DarkResidualBlock, in_channels=64, num_blocks=1)

        self.conv3 = conv_batch(64, 128, kernel_size=3, stride=2)
        self.residual_block2 = make_layer(DarkResidualBlock, in_channels=128, num_blocks=2)

        self.conv4 = conv_batch(128, 256, kernel_size=3, stride=2)
        self.residual_block3 = make_layer(DarkResidualBlock, in_channels=256, num_blocks=8)

        self.conv5 = conv_batch(256, 512, kernel_size=3, stride=2)
        self.residual_block4 = make_layer(DarkResidualBlock, in_channels=512, num_blocks=8)

        self.conv6 = conv_batch(512, 1024, kernel_size=3, stride=2)
        self.residual_block5 = make_layer(DarkResidualBlock, in_channels=1024, num_blocks=4)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.residual_block1(x)

        x = self.conv3(x)
        x = self.residual_block2(x)

        x = self.conv4(x)
        x = self.residual_block3(x)

        x = self.conv5(x)
        x = self.residual_block4(x)

        x = self.conv6(x)
        x = self.residual_block5(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [8]:
import numpy as np
import torch

# Function to load pre-trained weights using NumPy
def load_darknet_weights(model, weights_path):
    with open(weights_path, 'rb') as f:
        header = f.read(16)  # Skip the first 16 bytes (metadata)
        weights = np.fromfile(f, dtype=np.float32)  # Use NumPy to read binary weights

    ptr = 0
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            conv_layer = module

            # Check if the layer has biases and load them if they exist
            if conv_layer.bias is not None:
                num_biases = conv_layer.bias.numel()
                biases = torch.from_numpy(weights[ptr:ptr + num_biases]).view_as(conv_layer.bias.data)
                conv_layer.bias.data.copy_(biases)
                ptr += num_biases

            # Load the weights
            num_weights = conv_layer.weight.numel()
            conv_weights = torch.from_numpy(weights[ptr:ptr + num_weights]).view_as(conv_layer.weight.data)
            conv_layer.weight.data.copy_(conv_weights)
            ptr += num_weights


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Set the path to your pre-trained weights
weights_path = '/content/drive/MyDrive/darknet53.conv.74'

In [11]:
# Instantiate the model and load weights
model = Darknet53(num_classes=31)
load_darknet_weights(model, weights_path)

In [12]:
# Optional: Freeze early layers
for param in model.conv1.parameters():
    param.requires_grad = False

In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Blended/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [15]:
val_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Blended/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [16]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [17]:
# Training loop with time and additional metrics
num_epochs = 25
best_val_acc = 0.0
best_val_loss = float('inf')
best_precision = 0.0
best_recall = 0.0
best_f1 = 0.0

In [18]:
model = model.to(device)

In [22]:
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_predictions, average='macro', zero_division=1)
    recall = recall_score(all_labels, all_predictions, average='macro',zero_division=1)
    f1 = f1_score(all_labels, all_predictions, average='macro', zero_division=1)

    # Save the best model based on validation accuracy
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        best_val_loss = val_loss
        best_precision = precision
        best_recall = recall
        best_f1 = f1
        torch.save(model.state_dict(), '/content/drive/MyDrive/darknet53_Blended_model.pth')

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%, Precision: {precision:.4f}, '
          f'Recall: {recall:.4f}, F1 Score: {f1:.4f}')

end_time = time.time()
elapsed_time = end_time - start_time
minutes, seconds = divmod(elapsed_time, 60)

print(f"Training complete in {int(minutes)}m {int(seconds)}s")
print(f"Best val Acc: {best_val_acc:.4f}")
print(f"Best val Loss: {best_val_loss:.4f}")
print(f"Best val Precision: {best_precision:.4f}")
print(f"Best val Recall: {best_recall:.4f}")
print(f"Best val F1 Score: {best_f1:.4f}")

Epoch [1/25], Train Loss: 1.0139, Train Acc: 72.28%, Val Loss: 0.9248, Val Acc: 78.16%, Precision: 0.8243, Recall: 0.7807, F1 Score: 0.7463
Epoch [2/25], Train Loss: 0.7025, Train Acc: 80.53%, Val Loss: 0.6449, Val Acc: 82.47%, Precision: 0.8511, Recall: 0.8443, F1 Score: 0.8321
Epoch [3/25], Train Loss: 0.5808, Train Acc: 83.98%, Val Loss: 0.4025, Val Acc: 90.00%, Precision: 0.9044, Recall: 0.9070, F1 Score: 0.8909
Epoch [4/25], Train Loss: 0.4465, Train Acc: 87.51%, Val Loss: 0.4517, Val Acc: 89.25%, Precision: 0.9124, Recall: 0.9063, F1 Score: 0.9047
Epoch [5/25], Train Loss: 0.4102, Train Acc: 88.80%, Val Loss: 0.2959, Val Acc: 92.55%, Precision: 0.9353, Recall: 0.9342, F1 Score: 0.9330
Epoch [6/25], Train Loss: 0.3202, Train Acc: 91.26%, Val Loss: 0.3277, Val Acc: 92.52%, Precision: 0.9371, Recall: 0.9352, F1 Score: 0.9332
Epoch [7/25], Train Loss: 0.2955, Train Acc: 91.43%, Val Loss: 0.3167, Val Acc: 90.85%, Precision: 0.9210, Recall: 0.9214, F1 Score: 0.9164
Epoch [8/25], Train 

In [23]:
import os
print(os.listdir('/content/drive/MyDrive/'))  # Replace with your directory

['Colab Notebooks', 'Blended', 'darknet53.conv.74', 'data_224_g', 'malevis_train_val_224x224', 'darknet53_full_model.pth', 'darknet53_Blended_model.pth']


In [28]:
from PIL import Image
# Prediction function
def predict_image(image_path, model, class_names):
    model.eval()  # Set the model to evaluation mode

    # Image transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize image to 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load image and apply transformations
    image = Image.open(image_path).convert('RGB')  # Convert to RGB to ensure 3 channels
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move image to device
    image = image.to(device)

    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    # Get the class name
    predicted_class = class_names[preds.item()]
    return predicted_class

# Load the saved model
model = Darknet53(num_classes=31)  # Ensure num_classes matches your model
model.load_state_dict(torch.load('/content/drive/MyDrive/darknet53_Blended_model.pth', weights_only=False))  # Load the best saved model
model = model.to(device)

# Example usage
class_names = train_dataset.classes  # Get class names from the training dataset
image_path = '/content/drive/MyDrive/Blended/val/Expiro/b056acc9c32458f923a78007c44753a27139bc65resized_image.png'  # Replace with the path to your image
predicted_class = predict_image(image_path, model, class_names)
print(f"Predicted class: {predicted_class}")


Predicted class: Expiro
